# <center>Introduction to <img src="https://www.python.org/static/community_logos/python-logo-inkscape.svg" alt="Python" width=300/> and <img src="https://ipython.org/_static/IPy_header.png" alt="IPython" width=300/> using <img src="https://upload.wikimedia.org/wikipedia/commons/3/38/Jupyter_logo.svg" alt="Jupyter" width=75/></center>

# Import Modules and set Paths

In [1]:
# Basic Packages
from __future__ import division
import os
from datetime import datetime

# Web & file access
import requests
import io

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

%pylab --no-import-all
%matplotlib inline

import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context("talk")

import plotly.express as px
import plotly.graph_objects as go

from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap
# Next line can import all of plotnine, but may overwrite things? Better import each function/object you need
#from plotnine import *

# Data
import pandas as pd
import numpy as np
from pandas_datareader import data, wb

# GIS & maps
import geopandas as gpd
gp = gpd
import georasters as gr
import geoplot as gplt
import geoplot.crs as gcrs
import mapclassify as mc
import textwrap

# Data Munging
from itertools import product, combinations
import difflib
import pycountry
import geocoder
from geonamescache.mappers import country
mapper = country(from_key='name', to_key='iso3')
mapper2 = country(from_key='iso3', to_key='iso')
mapper3 = country(from_key='iso3', to_key='name')

# Regressions & Stats
from scipy.stats import norm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer, LineLocation

# Paths
pathout = './data/'

if not os.path.exists(pathout):
    os.mkdir(pathout)
    
pathgraphs = './graphs/'
if not os.path.exists(pathgraphs):
    os.mkdir(pathgraphs)

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


<div class="alert alert-block alert-info">
    <b>Note:</b>
</div>

<div class="alert alert-block alert-danger">
<b>Careful:</b> 
</div>

<div class="alert alert-block alert-warning">
<b>Exercise 1:</b>
</div>

In [ ]:
def my_xy_plot(dfin, 
               x='SP.POP.GROW', 
               y='ln_gdp_pc', 
               labelvar='iso3c', 
               dx=0.006125, 
               dy=0.006125, 
               xlogscale=False, 
               ylogscale=False,
               xlabel='Growth Rate of Population', 
               ylabel='Log[Income per capita in ' +  str(year) + ']',
               labels=False,
               xpct=False,
               ypct=False,
               OLS=False,
               OLSlinelabel='OLS',
               ssline=False,
               sslinelabel='45 Degree Line',
               filename='income-pop-growth.pdf',
               hue='region',
               hue_order=['East Asia & Pacific', 'Europe & Central Asia',
                          'Latin America & Caribbean', 'Middle East & North Africa',
                          'North America', 'South Asia', 'Sub-Saharan Africa'],
               style='incomeLevel', 
               style_order=['High Income', 'Upper Middle Income', 'Lower Middle Income', 'Low Income'],
               palette=None,
               size=None,
               sizes=None,
               s=50,
               legend_fontsize=10,
               label_font_size=12,
               xlabel_font_size=12,
               ylabel_font_size=12,
               weight=None,
               save=True):
    '''
    Plot the association between x and y in the dataframe using labelvar for labels.
    
    Parameters:
    - dfin: DataFrame containing the data to plot.
    - x: Column name to be used for x-axis values.
    - y: Column name to be used for y-axis values.
    - labelvar: Column name for the labels of the points.
    - dx: Horizontal adjustment for the labels.
    - dy: Vertical adjustment for the labels.
    - xlogscale: Boolean to use logarithmic scale for x-axis.
    - ylogscale: Boolean to use logarithmic scale for y-axis.
    - xlabel: Label for x-axis.
    - ylabel: Label for y-axis.
    - labels: Boolean to display labels.
    - xpct: Boolean to display x-axis values as percentages.
    - ypct: Boolean to display y-axis values as percentages.
    - OLS: Boolean to add an Ordinary Least Squares regression line.
    - OLSlinelabel: Label for the OLS regression line.
    - ssline: Boolean to add a 45-degree line.
    - sslinelabel: Label for the 45-degree line.
    - filename: Filename to save the plot.
    - hue: Column name for color encoding.
    - hue_order: Order of categories for the hue.
    - style: Column name for style encoding.
    - style_order: Order of categories for the style.
    - palette: Color palette.
    - size: Column name for size encoding.
    - sizes: Sizes of the points.
    - s: Size of the points.
    - legend_fontsize: Font size of the legend.
    - label_font_size: Font size of the labels.
    - xlabel_font_size: Font size of the x-axis label.
    - ylabel_font_size: Font size of the y-axis label.
    - weight: Font weight for labels.
    - save: Boolean to save the plot.
    
    Returns:
    - fig: The resulting plot figure.

    Example usage
    - Assuming 'df' is a pandas DataFrame with the relevant data:
    fig = my_xy_plot(df, x='population_growth', y='gdp_per_capita', labelvar='country_code', labels=True, OLS=True)

    '''
    
    # Set plot size and context
    sns.set(rc={'figure.figsize': (11.7, 8.27)})
    sns.set_context("talk")
    
    # Create a copy of the dataframe and drop rows with NaNs in x or y
    df = dfin.copy()
    df = df.dropna(subset=[x, y]).reset_index(drop=True)
    
    # Initialize plot
    fig, ax = plt.subplots()
    
    # Create scatterplot
    sns.scatterplot(x=x, y=y, data=df, ax=ax, 
                    hue=hue,
                    hue_order=hue_order,
                    alpha=1, 
                    style=style, 
                    style_order=style_order,
                    palette=palette,
                    size=size,
                    sizes=sizes,
                    s=s)
    
    # Add OLS regression line if specified
    if OLS:
        sns.regplot(x=x, y=y, data=df, ax=ax, label=OLSlinelabel, scatter=False)
    
    # Add 45-degree line if specified
    if ssline:
        ax.plot([df[x].min() * 0.99, df[x].max() * 1.01], 
                [df[x].min() * 0.99, df[x].max() * 1.01], 
                c='r', label=sslinelabel)
    
    # Add labels to points if specified
    if labels:
        movex = df[x].mean() * dx
        movey = df[y].mean() * dy
        for line in range(df.shape[0]):
            ax.text(df[x][line] + movex, df[y][line] + movey, 
                    df[labelvar][line], horizontalalignment='left', 
                    fontsize=label_font_size, color='black')
    
    # Set axis labels with specified font sizes and weights
    ax.set_xlabel(xlabel, weight=weight, fontsize=xlabel_font_size)
    ax.set_ylabel(ylabel, weight=weight, fontsize=ylabel_font_size)
    
    # Format x-axis as percentage if specified
    if xpct:
        fmt = '%.0f%%' 
        xticks = mtick.FormatStrFormatter(fmt)
        ax.xaxis.set_major_formatter(xticks)
    
    # Format y-axis as percentage if specified
    if ypct:
        fmt = '%.0f%%'
        yticks = mtick.FormatStrFormatter(fmt)
        ax.yaxis.set_major_formatter(yticks)
    
    # Set logarithmic scale for x-axis if specified
    if xlogscale:
        ax.set(xscale="log")
    
    # Set logarithmic scale for y-axis if specified
    if ylogscale:
        ax.set(yscale="log")
    
    # Customize legend
    handles, labels = ax.get_legend_handles_labels()
    handles = np.array(handles)
    labels = np.array(labels)
    handles = list(handles[(labels != hue) & (labels != style) & (labels != size)])
    labels = list(labels[(labels != hue) & (labels != style) & (labels != size)])
    ax.legend(handles=handles, labels=labels, fontsize=legend_fontsize)
    
    # Save plot if specified
    if save:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
    
    return fig


In [181]:
def my_xy_line_plot(dfin, 
                   x='SP.POP.GROW', 
                   y='ln_gdp_pc', 
                   labelvar='iso3c', 
                   dx=0.006125, 
                   dy=0.006125, 
                   xlogscale=False, 
                   ylogscale=False,
                   xlabel='Growth Rate of Population', 
                   ylabel='Log[Income per capita in ' + str(year) + ']',
                   labels=False,
                   xpct=False,
                   ypct=False,
                   OLS=False,
                   OLSlinelabel='OLS',
                   ssline=False,
                   sslinelabel='45 Degree Line',
                   filename='income-pop-growth.pdf',
                   hue='region',
                   hue_order=['East Asia & Pacific', 'Europe & Central Asia',
                              'Latin America & Caribbean', 'Middle East & North Africa',
                              'North America', 'South Asia', 'Sub-Saharan Africa'],
                   style='incomeLevel', 
                   style_order=['High Income', 'Upper Middle Income', 'Lower Middle Income', 'Low Income'],
                   palette=None,
                   legend_fontsize=10,
                   label_font_size=12,
                   save=True):
    '''
    Plot the association between x and y in the dataframe using labelvar for labels.
    
    Parameters:
    - dfin: DataFrame containing the data to plot.
    - x: Column name to be used for x-axis values.
    - y: Column name to be used for y-axis values.
    - labelvar: Column name for the labels of the points.
    - dx: Horizontal adjustment for the labels.
    - dy: Vertical adjustment for the labels.
    - xlogscale: Boolean to use logarithmic scale for x-axis.
    - ylogscale: Boolean to use logarithmic scale for y-axis.
    - xlabel: Label for x-axis.
    - ylabel: Label for y-axis.
    - labels: Boolean to display labels.
    - xpct: Boolean to display x-axis values as percentages.
    - ypct: Boolean to display y-axis values as percentages.
    - OLS: Boolean to add an Ordinary Least Squares regression line.
    - OLSlinelabel: Label for the OLS regression line.
    - ssline: Boolean to add a 45-degree line.
    - sslinelabel: Label for the 45-degree line.
    - filename: Filename to save the plot.
    - hue: Column name for color encoding.
    - hue_order: Order of categories for the hue.
    - style: Column name for style encoding.
    - style_order: Order of categories for the style.
    - palette: Color palette.
    - legend_fontsize: Font size of the legend.
    - label_font_size: Font size of the labels.
    - save: Boolean to save the plot.
    
    Returns:
    - fig: The resulting plot figure.
    
    Example usage:
    Assuming 'df' is a pandas DataFrame with the relevant data:
    fig = my_xy_line_plot(df, x='population_growth', y='gdp_per_capita', labelvar='country_code', labels=True, OLS=True)
    '''
    
    # Set plot size and context
    sns.set(rc={'figure.figsize': (11.7, 8.27)})
    sns.set_context("talk")
    
    # Create a copy of the dataframe and drop rows with NaNs in x or y
    df = dfin.copy()
    df = df.dropna(subset=[x, y]).reset_index(drop=True)
    
    # Initialize plot
    fig, ax = plt.subplots()
    
    # Create line plot
    sns.lineplot(x=x, y=y, data=df, ax=ax, 
                 hue=hue,
                 hue_order=hue_order,
                 alpha=1, 
                 style=style, 
                 style_order=style_order,
                 palette=palette)
    
    # Add OLS regression line if specified
    if OLS:
        sns.regplot(x=x, y=y, data=df, ax=ax, label=OLSlinelabel, scatter=False)
    
    # Add 45-degree line if specified
    if ssline:
        ax.plot([df[x].min() * 0.99, df[x].max() * 1.01], 
                [df[x].min() * 0.99, df[x].max() * 1.01], 
                c='r', label=sslinelabel)
    
    # Add labels to points if specified
    if labels:
        movex = df[x].mean() * dx
        movey = df[y].mean() * dy
        for line in range(df.shape[0]):
            ax.text(df[x][line] + movex, df[y][line] + movey, 
                    df[labelvar][line], horizontalalignment='left', 
                    fontsize=label_font_size, color='black')
    
    # Set axis labels
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
    # Format x-axis as percentage if specified
    if xpct:
        fmt = '%.0f%%'
        xticks = mtick.FormatStrFormatter(fmt)
        ax.xaxis.set_major_formatter(xticks)
    
    # Format y-axis as percentage if specified
    if ypct:
        fmt = '%.0f%%'
        yticks = mtick.FormatStrFormatter(fmt)
        ax.yaxis.set_major_formatter(yticks)
    
    # Set logarithmic scale for x-axis if specified
    if xlogscale:
        ax.set(xscale="log")
    
    # Set logarithmic scale for y-axis if specified
    if ylogscale:
        ax.set(yscale="log")
    
    # Customize legend
    handles, labels = ax.get_legend_handles_labels()
    handles = np.array(handles)
    labels = np.array(labels)
    handles = list(handles[(labels != 'region') & (labels != 'incomeLevel')])
    labels = list(labels[(labels != 'region') & (labels != 'incomeLevel')])
    ax.legend(handles=handles, labels=labels, fontsize=legend_fontsize)
    
    # Save plot if specified
    if save:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
    
    return fig

In [ ]:
# Functions for plotting
def center_wrap(text, cwidth=32, **kw):
    '''Center Text (to be used in legend)'''
    lines = text
    #lines = textwrap.wrap(text, **kw)
    return "\n".join(line.center(cwidth) for line in lines)

def MyChoropleth(mydf, 
                 myfile='', 
                 myvar='',
                 mylegend='', 
                 k=5,
                 extent=[-180, -90, 180, 90],
                 figsize=(24, 16),
                 bbox_to_anchor=(0.2, 0.5),
                 fontsize=12, 
                 title_fontsize=12,
                 edgecolor='white', 
                 facecolor='lightgray',
                 scheme='FisherJenks', 
                 bins=None, 
                 pct=None,
                 legend=True, 
                 legend_labels=None, 
                 legend_kwargs=None,
                 legend_values=None,
                 bold=True,
                 rasterized=False,
                 save=True,
                 percent=False,
                 rn=0,
                 cmap='Reds',
                 reverse=False,
                 **kwargs):
    '''
    Create a choropleth map with the specified parameters.
    
    Parameters:
    - mydf: DataFrame containing the geographical data to plot.
    - myfile: File name for saving the plot.
    - myvar: Column name for the variable to be mapped.
    - mylegend: Title for the legend.
    - k: Number of classes for the choropleth.
    - extent: Geographical extent of the plot. Example: [min_longitude, min_latitude, max_longitude, max_latitude]
    - figsize: Size of the plot figure.
    - bbox_to_anchor: Bounding box for the legend.
    - fontsize: Font size for the legend text.
    - title_fontsize: Font size for the legend title.
    - edgecolor: Color of the edges of the polygons.
    - facecolor: Background color of the plot.
    - scheme: Classification scheme for the choropleth. Possible values: 'EqualInterval', 'Quantiles', 'BoxPlot', 'FisherJenks', 'FisherJenksSampled', 'HeadTailBreaks', 'JenksCaspall', 'JenksCaspallForced', 'JenksCaspallSampled', 'KClassifiers', 'Percentiles', 'UserDefined'
    - bins: Bins for user-defined classification scheme.
    - pct: Percentiles for percentile-based classification scheme.
    - legend: Boolean to display the legend.
    - legend_labels: Custom labels for the legend.
    - legend_kwargs: Additional keyword arguments for the legend.
    - legend_values: Custom values for the legend.
    - bold: Boolean to make the legend title bold.
    - rasterized: Boolean to rasterize the plot.
    - save: Boolean to save the plot.
    - percent: Boolean to format the legend labels as percentages.
    - rn: Number of decimal places for the legend labels.
    - cmap: Color map for the choropleth.
    - reverse: Boolean to reverse the order of the legend labels.
    - kwargs: Additional keyword arguments for the plot.
    
    Returns:
    - ax: The resulting plot axis.
    
    Example usage:
    Assuming 'gdf' is a GeoDataFrame with the relevant data:
    ax = MyChoropleth(gdf, myfile='map', myvar='population_density', mylegend='Population Density', k=5)
    '''
    
    # Determine classification scheme
    if scheme == 'EqualInterval':
        scheme = mc.EqualInterval(mydf[myvar], k=k)
    elif scheme == 'Quantiles':
        scheme = mc.Quantiles(mydf[myvar], k=k)
    elif scheme == 'BoxPlot':
        scheme = mc.BoxPlot(mydf[myvar], k=k)
    elif scheme == 'FisherJenks':
        scheme = mc.FisherJenks(mydf[myvar], k=k)
    elif scheme == 'FisherJenksSampled':
        scheme = mc.FisherJenksSampled(mydf[myvar], k=k)
    elif scheme == 'HeadTailBreaks':
        scheme = mc.HeadTailBreaks(mydf[myvar], k=k)
    elif scheme == 'JenksCaspall':
        scheme = mc.JenksCaspall(mydf[myvar], k=k)
    elif scheme == 'JenksCaspallForced':
        scheme = mc.JenksCaspallForced(mydf[myvar], k=k)
    elif scheme == 'JenksCaspallSampled':
        scheme = mc.JenksCaspallSampled(mydf[myvar], k=k)
    elif scheme == 'KClassifiers':
        scheme = mc.KClassifiers(mydf[myvar], k=k)
    elif scheme == 'Percentiles':
        scheme = mc.Percentiles(mydf[myvar], pct=pct)
    elif scheme == 'UserDefined':
        scheme = mc.UserDefined(mydf[myvar], bins=bins)
    
    # Configure legend settings if legend is to be displayed
    if legend:
        legend_kwargs = {'bbox_to_anchor': bbox_to_anchor,
                         'frameon': True,
                         'title': mylegend,
                         'fontsize': fontsize,
                         'title_fontsize': title_fontsize}
        
        if legend_labels is None:
            # Format legend labels
            upper_bounds = scheme.bins
            bounds = []
            for index, upper_bound in enumerate(upper_bounds):
                if index == 0:
                    lower_bound = mydf[myvar].min()
                else:
                    lower_bound = upper_bounds[index - 1]
                # Format numerical legend
                if percent:
                    bound = f'{lower_bound:.{rn}%} - {upper_bound:.{rn}%}'.format(width=rn)
                else:
                    bound = f'{float(lower_bound):,.{rn}f} - {float(upper_bound):,.{rn}f}'.format(width=rn)
                bounds.append(bound)
            legend_labels = bounds
        
        # Reverse legend labels if specified
        if reverse:
            legend_labels = legend_labels[::-1]
            legend_values = list(range(len(legend_labels)))[::-1]
    
    # Plot choropleth
    ax = gplt.choropleth(
        mydf, hue=myvar, projection=gcrs.PlateCarree(central_longitude=0.0),
        edgecolor=edgecolor, linewidth=1,
        cmap=cmap, legend=legend,
        scheme=scheme,
        legend_kwargs=legend_kwargs,
        legend_labels=legend_labels,
        legend_values=legend_values,
        figsize=figsize,
        rasterized=rasterized,
        **kwargs
    )
    
    # Plot country borders
    gplt.polyplot(
        gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')),
        projection=gcrs.PlateCarree(central_longitude=0.0),
        edgecolor=edgecolor, facecolor=facecolor,
        ax=ax,
        rasterized=rasterized,
        extent=extent
    )
    
    # Make legend title bold if specified
    if bold:
        ax.get_legend().get_title().set_fontweight('bold')
    
    # Save plot if specified
    if save:
        plt.savefig(myfile + '_' + myvar.replace(' ', '_') + '.jpg', dpi=300, bbox_inches='tight')
    
    return ax

In [ ]:
def my_kde_plots_gdppc(dataset='pwt', myyear=2010):
    '''
    This function plots the KDE-plot of log income per capita, where
    
    dataset: is either pwt or wdi (you need to ensure the variable 'gdp_pc' exists)
    myyear: is the year you want the distribution
    '''
    df = eval(dataset + '.loc[' + dataset +'.year==' + str(myyear) + '].copy().reset_index(drop=True)')
    # Plot
    fig, ax = plt.subplots()
    sns.kdeplot(df.gdp_pc.apply(np.log), ax=ax, fill=True, label=str(myyear), linewidth=2)
    ax.set_xlabel('Log[Income per capita, '+ dataset.upper() + ']')
    ax.set_ylabel('Density of Countries')
    ax.legend()
    plt.savefig(pathgraphs + dataset +'_gdppc_' + str(myyear) + '.pdf', dpi=300, bbox_inches='tight')
    plt.draw()
    return "Done"

In [ ]:
def my_kde_plots_gdppc_joint(dataset='pwt', myyears=[2000, 2020]):
    '''
    This function plots the KDE-plot of log income per capita in two years, where
    
    dataset: is either pwt or wdi (you need to ensure the variable 'gdp_pc' exists)
    myyears: is a list of years
    '''
    df = eval(dataset + '.loc[' + dataset +'.year.isin(myyears)].copy().reset_index(drop=True)')
    # Plot
    fig, ax = plt.subplots()
    t in range(len(myyears)):
        sns.kdeplot(df.loc[df.year==myyears[t]].gdp_pc.apply(np.log), ax=ax, fill=True, label=str(myyears[t]), linewidth=2, color='r')
    ax.set_xlabel('Log[Income per capita, '+ dataset.upper() + ']')
    ax.set_ylabel('Density of Countries')
    ax.legend()
    plt.savefig(pathgraphs + dataset +'_gdppc_' + str(myyears[0]) + '_' + str(myyears[-1]) + '.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(pathgraphs + dataset +'_gdppc_' + str(myyears[0]) + '_' + str(myyears[-1]) + '.png', dpi=300, bbox_inches='tight')
    plt.draw()
    return "Done"

In [ ]:
def my_kde_plots(dataset='pwt', myyear=2010, myvar='gdp_pc_e', myvar_label='Income per capita'):
    '''
    This function plots the KDE-plot of myvar in a given year.
    
    Parameters:
    - dataset: The dataset to use ('pwt' or 'wdi'). Ensure the variable 'gdp_pc' exists in the dataset.
    - myyear: The year for which the distribution is to be plotted.
    - myvar: The variable to plot the KDE for.
    - myvar_label: The label for the x-axis.
    
    Returns:
    - str: "Done" when the plot is completed and saved.
    
    Example usage:
    Assuming 'pwt' or 'wdi' is a DataFrame with the relevant data:
    result = my_kde_plots(dataset='pwt', myyear=2010, myvar='gdp_pc_e', myvar_label='Income per capita')
    '''
    
    # Evaluate the dataset and filter it for the given year
    df = eval(dataset + '.loc[' + dataset + '.year==' + str(myyear) + '].copy().reset_index(drop=True)')
    
    # Initialize plot
    fig, ax = plt.subplots()
    
    # Plot KDE
    sns.kdeplot(df[myvar], ax=ax, fill=True, label=str(myyear), linewidth=2)
    
    # Set axis labels
    ax.set_xlabel(myvar_label)
    ax.set_ylabel('Density of Countries')
    
    # Add legend
    ax.legend()
    
    # Save plot
    plt.savefig(dataset + '_gdppc_' + str(myyear) + '.pdf', dpi=300, bbox_inches='tight')
    
    # Draw plot
    plt.draw()
    
    return "Done"

In [ ]:
def my_kde_plots_joint(dataset='pwt', myyears=[2000, 2010, 2020], myvar='gdp_pc_e', myvar_label='Income per capita'):
    '''
    This function plots the KDE-plot of myvar for all years in the given range.
    
    Parameters:
    - dataset: The dataset to use ('pwt' or 'wdi'). Ensure the variable 'gdp_pc' exists in the dataset.
    - myyears: A list of years to plot the KDE for.
    - myvar: The variable to plot the KDE for.
    - myvar_label: The label for the x-axis.
    
    Returns:
    - str: "Done" when the plot is completed and saved.
    
    Example usage:
    Assuming 'pwt' or 'wdi' is a DataFrame with the relevant data:
    result = my_kde_plots_joint(dataset='pwt', myyears=[2000, 2010, 2020], myvar='gdp_pc_e', myvar_label='Income per capita')
    '''
    
    # Evaluate the dataset and filter it for the given years
    df = eval(dataset + '.loc[' + dataset + '.year.isin(myyears)].copy().reset_index(drop=True)')
    
    # Initialize plot
    mycolors = sns.color_palette("Paired", n_colors=len(myyears))
    fig, ax = plt.subplots()
    
    # Plot KDE for each year
    for t in range(len(myyears)):
        sns.kdeplot(df.loc[df.year == myyears[t], myvar], ax=ax, fill=True, label=str(myyears[t]), linewidth=2, color=mycolors[t])
    
    # Set axis labels
    ax.set_xlabel(myvar_label)
    ax.set_ylabel('Density of Countries')
    
    # Add legend
    ax.legend()
    
    # Save plot
    plt.savefig(dataset + '-' + myvar + '_' + str(myyears[0]) + '-' + str(myyears[-1]) + '.pdf', dpi=300, bbox_inches='tight')
    
    # Draw plot
    plt.draw()
    
    return "Done"

Notebook written by [Ömer Özak](http://omerozak.com) for his students in Economics at [Southern Methodist University](http://www.smu.edu). Feel free to use, distribute, or contribute.

[<center><img src="https://github.com/measuring-culture/Expanding-Measurement-Culture-Facebook-JRSI/blob/main/pics/SMUlogowWordmarkRB.jpg?raw=true" width="250"></center>](http://omerozak.com)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4e22fe6b-8ce8-4ee0-aabf-b3ced0204580' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>